<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/Neuron/blob/main/Tutorial_3_Agent_Communication_Making_Agents_Talk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Tutorial 3: Agent Communication - Making Agents Talk

In the previous tutorials, you built individual agents that can process messages and remember conversations. Now we're going to connect multiple agents together so they can communicate with each other through a centralized message bus.

## What you'll build:
- A SynapticBus system that routes messages between agents  
- Three agents with different personalities (friendly, analytical, helpful)  
- Direct messaging and group conversation capabilities  
- Topic-based subscriptions for broadcast messages  
- Real-time visualization of communication patterns  

## Why this matters:
Most AI systems in production use multiple agents working together. Chat platforms, recommendation systems, and autonomous vehicles all rely on agents that can discover, communicate, and coordinate with each other. This tutorial teaches you those foundational patterns.

## By the end, you'll understand:
- How to build scalable multi-agent architectures  
- Message routing and priority handling  
- Agent discovery and registration patterns  
- How personality affects agent interactions  
- Debugging and monitoring distributed agent systems  

Let's start building...

In [1]:
import uuid
import time
import threading
import queue
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Set, Callable
from enum import Enum
import json
from collections import defaultdict, deque

In [9]:
# Import our foundation from previous tutorials
AgentID = str
MessageID = str

class MessagePriority(Enum):
    LOW = 1
    NORMAL = 2
    HIGH = 3
    URGENT = 4

@dataclass
class Message:
    id: MessageID
    sender: AgentID
    recipients: List[AgentID]
    content: Any
    priority: MessagePriority = MessagePriority.NORMAL
    metadata: Dict[str, Any] = field(default_factory=dict)
    created_at: float = field(default_factory=time.time)

    @classmethod
    def create(cls, sender: AgentID, recipients: List[AgentID], content: Any,
               priority: MessagePriority = MessagePriority.NORMAL) -> 'Message':
        return cls(
            id=str(uuid.uuid4()),
            sender=sender,
            recipients=recipients,
            content=content,
            priority=priority
        )

class MessageType(Enum):
    """
    Different types of messages for different communication patterns

    This helps agents understand what kind of message they're receiving
    and how they should respond to it.
    """
    REQUEST = "request"      # Asking for something (expects response)
    RESPONSE = "response"    # Answering a request
    BROADCAST = "broadcast"  # Information for everyone
    NOTIFICATION = "notification"  # FYI message (no response expected)
    COMMAND = "command"      # Instruction to do something
    QUERY = "query"          # Question about data/state

class AgentMetrics:
    """Simple metrics tracking for agents"""
    def __init__(self):
        self.message_count = 0
        self.processing_time = 0.0
        self.error_count = 0
        self.last_active = None

    def update_processing_time(self, time_delta: float):
        self.processing_time += time_delta
        self.last_active = time.time()

    def increment_message_count(self):
        self.message_count += 1
        self.last_active = time.time()

class SimpleWorkingMemory:
    """
    Simple memory system for this tutorial

    This provides basic memory functionality without external dependencies.
    """
    def __init__(self, capacity: int = 10):
        self.capacity = capacity
        self._items = []
        self._lock = threading.Lock()

    def store(self, content: Any, importance: float = 1.0, metadata: Dict = None):
        """Store an item in memory"""
        with self._lock:
            self._items.insert(0, content)
            if len(self._items) > self.capacity:
                self._items = self._items[:self.capacity]
        return str(uuid.uuid4())

    def get_context(self, max_items: int = 5):
        """Get recent context items"""
        with self._lock:
            return self._items[:max_items]

    def size(self):
        """Get current memory size"""
        return len(self._items)

class SynapticBus:
    """
    Central message bus for agent communication

    This is like a postal service for agents - it handles routing messages
    between agents, manages queues, and ensures reliable delivery.

    Think of it as the nervous system that connects all the agents together.
    """

    def __init__(self, max_queue_size: int = 1000):
        # Core routing infrastructure
        self._agents: Dict[AgentID, 'BaseAgent'] = {}  # Registered agents
        self._message_queues: Dict[AgentID, queue.PriorityQueue] = {}  # Agent message queues
        self._routing_table: Dict[str, Set[AgentID]] = defaultdict(set)  # Topic-based routing

        # Message processing
        self.max_queue_size = max_queue_size
        self._message_history: deque = deque(maxlen=1000)  # Recent message history
        self._delivery_stats: Dict[str, int] = defaultdict(int)  # Delivery statistics

        # Threading and lifecycle
        self._stop_event = threading.Event()
        self._router_thread = None
        self._running = False
        self._lock = threading.Lock()  # Thread safety for registration operations

        print(f"🚌 SynapticBus initialized with max queue size: {max_queue_size}")

    def start(self):
        """
        Start the message bus routing system

        This begins the background routing process that moves messages
        from senders to receivers based on priorities and routing rules.
        """
        if self._running:
            print("⚠️  SynapticBus is already running")
            return

        self._stop_event.clear()
        self._router_thread = threading.Thread(
            target=self._routing_loop,
            daemon=True,
            name="SynapticBus-Router"
        )
        self._router_thread.start()
        self._running = True
        print("▶️  SynapticBus started routing messages")

    def stop(self):
        """Stop the message bus and clean up resources"""
        if not self._running:
            return

        self._stop_event.set()
        if self._router_thread:
            self._router_thread.join(timeout=2.0)
            if self._router_thread.is_alive():
                print("⚠️  Warning: SynapticBus router didn't stop cleanly")

        self._running = False
        print("⏹️  SynapticBus stopped")

    def register_agent(self, agent: 'BaseAgent') -> bool:
        """
        Register an agent with the message bus

        This allows the agent to send and receive messages through the bus.
        Each agent gets its own priority queue for incoming messages.
        """
        with self._lock:
            if agent.id in self._agents:
                print(f"⚠️  Agent {agent.name} ({agent.id[:8]}...) already registered")
                return False

            # Register the agent and create its message queue
            self._agents[agent.id] = agent
            self._message_queues[agent.id] = queue.PriorityQueue(maxsize=self.max_queue_size)

            print(f"📝 Registered agent: {agent.name} ({agent.id[:8]}...)")
            return True

    def unregister_agent(self, agent_id: AgentID) -> bool:
        """Remove an agent from the message bus"""
        with self._lock:
            if agent_id not in self._agents:
                return False

            agent_name = self._agents[agent_id].name
            del self._agents[agent_id]
            del self._message_queues[agent_id]

            # Remove from routing tables
            for topic_agents in self._routing_table.values():
                topic_agents.discard(agent_id)

            print(f"📝 Unregistered agent: {agent_name} ({agent_id[:8]}...)")
            return True

    def subscribe_to_topic(self, agent_id: AgentID, topic: str):
        """
        Subscribe an agent to a topic for broadcast messages

        Topics allow agents to receive messages about specific subjects
        without needing to know who else is interested.
        """
        with self._lock:
            if agent_id in self._agents:
                self._routing_table[topic].add(agent_id)
                agent_name = self._agents[agent_id].name
                print(f"📢 Agent {agent_name} subscribed to topic: '{topic}'")

    def unsubscribe_from_topic(self, agent_id: AgentID, topic: str):
        """Remove an agent's subscription to a topic"""
        with self._lock:
            self._routing_table[topic].discard(agent_id)
            if agent_id in self._agents:
                agent_name = self._agents[agent_id].name
                print(f"📢 Agent {agent_name} unsubscribed from topic: '{topic}'")

    def send_message(self, message: Message) -> bool:
        """
        Send a message through the bus

        This is the core routing function - it takes a message and
        delivers it to all specified recipients based on their queues.
        """
        if not self._running:
            print("⚠️  SynapticBus not running - message not sent")
            return False

        # Track message in history
        self._message_history.append({
            'id': message.id,
            'sender': message.sender,
            'recipients': message.recipients,
            'content': str(message.content)[:100],
            'priority': message.priority.name,
            'timestamp': message.created_at
        })

        delivered_count = 0

        # Deliver to each recipient
        for recipient_id in message.recipients:
            if recipient_id in self._message_queues:
                try:
                    # Use priority for queue ordering (lower numbers = higher priority)
                    priority_value = 5 - message.priority.value  # Invert so URGENT=4 becomes priority 1

                    self._message_queues[recipient_id].put(
                        (priority_value, message.created_at, message),
                        timeout=0.1
                    )
                    delivered_count += 1
                    self._delivery_stats['delivered'] += 1

                except queue.Full:
                    print(f"⚠️  Message queue full for agent {recipient_id[:8]}... - dropping message")
                    self._delivery_stats['dropped'] += 1
            else:
                print(f"⚠️  Unknown recipient: {recipient_id[:8]}...")
                self._delivery_stats['unknown_recipient'] += 1

        if delivered_count > 0:
            sender_name = "Unknown"
            if message.sender in self._agents:
                sender_name = self._agents[message.sender].name

            print(f"📬 Routed message from {sender_name} to {delivered_count} recipient(s)")

        return delivered_count > 0

    def broadcast_to_topic(self, sender: AgentID, topic: str, content: Any,
                          priority: MessagePriority = MessagePriority.NORMAL) -> int:
        """
        Broadcast a message to all agents subscribed to a topic

        This is useful for announcements, notifications, or any message
        that multiple agents might be interested in.
        """
        subscribers = list(self._routing_table[topic])

        if not subscribers:
            print(f"📢 No subscribers for topic '{topic}'")
            return 0

        message = Message.create(
            sender=sender,
            recipients=subscribers,
            content=content,
            priority=priority
        )
        message.metadata['message_type'] = MessageType.BROADCAST.value
        message.metadata['topic'] = topic

        success = self.send_message(message)

        if success:
            sender_name = "Unknown"
            if sender in self._agents:
                sender_name = self._agents[sender].name
            print(f"📢 {sender_name} broadcast to topic '{topic}': {len(subscribers)} subscribers")

        return len(subscribers) if success else 0

    def _routing_loop(self):
        """
        Main routing loop - processes message queues for all agents

        This continuously moves messages from agent queues to the
        actual agents, respecting priorities and delivery order.
        """
        print("🔄 SynapticBus routing loop started")

        while not self._stop_event.is_set():
            try:
                # Process messages for each agent
                for agent_id, message_queue in self._message_queues.items():
                    if agent_id in self._agents:
                        agent = self._agents[agent_id]

                        # Try to get a message for this agent
                        try:
                            priority_value, timestamp, message = message_queue.get_nowait()

                            # Deliver the message to the agent
                            agent.receive_message(message)
                            message_queue.task_done()

                        except queue.Empty:
                            # No messages for this agent right now
                            continue

                # Small delay to prevent CPU spinning
                time.sleep(0.01)

            except Exception as e:
                print(f"❌ Error in routing loop: {e}")
                time.sleep(0.1)

        print("🛑 SynapticBus routing loop stopped")

    def get_agent_info(self, agent_id: AgentID) -> Optional[Dict[str, Any]]:
        """Get information about a registered agent"""
        if agent_id in self._agents:
            agent = self._agents[agent_id]
            queue_size = self._message_queues[agent_id].qsize()

            return {
                'id': agent.id,
                'name': agent.name,
                'queue_size': queue_size,
                'subscribed_topics': [topic for topic, agents in self._routing_table.items()
                                    if agent_id in agents]
            }
        return None

    def get_system_stats(self) -> Dict[str, Any]:
        """Get comprehensive statistics about the message bus"""
        total_queue_size = sum(q.qsize() for q in self._message_queues.values())

        return {
            'running': self._running,
            'registered_agents': len(self._agents),
            'total_queue_size': total_queue_size,
            'delivery_stats': dict(self._delivery_stats),
            'topics': {topic: len(agents) for topic, agents in self._routing_table.items()},
            'recent_messages': len(self._message_history)
        }

    def get_recent_messages(self, limit: int = 10) -> List[Dict[str, Any]]:
        """Get recent message history for debugging"""
        return list(self._message_history)[-limit:]

class BaseAgent(ABC):
    """Enhanced BaseAgent that can communicate through SynapticBus"""

    def __init__(self, agent_id: Optional[AgentID] = None, name: str = "",
                 memory_capacity: int = 10, bus: Optional[SynapticBus] = None):
        # Core identification
        self.id = agent_id or str(uuid.uuid4())
        self.name = name or self.__class__.__name__

        # Message processing infrastructure
        self._message_queue = queue.Queue()
        self._stop_event = threading.Event()
        self._processing_thread = None

        # Communication system - NEW!
        self.bus = bus
        self._subscribed_topics: Set[str] = set()

        # Memory system (simplified version for this tutorial)
        self.working_memory = SimpleWorkingMemory(capacity=memory_capacity)

        # Monitoring and metrics
        self._metrics = AgentMetrics()
        self._running = False

        print(f"🤖 Initialized agent: {self.name} ({self.id[:8]}...)")

        # Auto-register with bus if provided
        if self.bus:
            self.bus.register_agent(self)

    def connect_to_bus(self, bus: SynapticBus):
        """Connect this agent to a message bus"""
        self.bus = bus
        self.bus.register_agent(self)
        print(f"🔌 Agent {self.name} connected to SynapticBus")

    def subscribe_to_topic(self, topic: str):
        """Subscribe to a topic for broadcast messages"""
        if self.bus:
            self.bus.subscribe_to_topic(self.id, topic)
            self._subscribed_topics.add(topic)

    def unsubscribe_from_topic(self, topic: str):
        """Unsubscribe from a topic"""
        if self.bus:
            self.bus.unsubscribe_from_topic(self.id, topic)
            self._subscribed_topics.discard(topic)

    def send_message_via_bus(self, recipients: List[AgentID], content: Any,
                            priority: MessagePriority = MessagePriority.NORMAL,
                            message_type: MessageType = MessageType.REQUEST) -> Optional[Message]:
        """
        Send a message through the SynapticBus

        This is the new way agents communicate - through the centralized bus
        rather than directly to each other.
        """
        if not self.bus:
            print(f"⚠️  Agent {self.name} not connected to bus - cannot send message")
            return None

        message = Message.create(
            sender=self.id,
            recipients=recipients,
            content=content,
            priority=priority
        )
        message.metadata['message_type'] = message_type.value

        success = self.bus.send_message(message)
        if success:
            self._metrics.increment_message_count()
            print(f"📤 Agent {self.name} sent via bus: {str(content)[:50]}{'...' if len(str(content)) > 50 else ''}")

        return message if success else None

    def broadcast_to_topic(self, topic: str, content: Any,
                          priority: MessagePriority = MessagePriority.NORMAL) -> int:
        """Broadcast a message to all subscribers of a topic"""
        if not self.bus:
            print(f"⚠️  Agent {self.name} not connected to bus - cannot broadcast")
            return 0

        return self.bus.broadcast_to_topic(self.id, topic, content, priority)

    def start(self):
        if self._running:
            print(f"⚠️  Agent {self.name} is already running")
            return

        self._stop_event.clear()
        self._processing_thread = threading.Thread(
            target=self._processing_loop,
            daemon=True,
            name=f"Agent-{self.name}"
        )
        self._processing_thread.start()
        self._running = True
        print(f"▶️  Agent {self.name} started")

    def stop(self):
        if not self._running:
            return

        self._stop_event.set()

        if self._processing_thread:
            self._processing_thread.join(timeout=2.0)
            if self._processing_thread.is_alive():
                print(f"⚠️  Warning: Agent {self.name} thread didn't stop cleanly")

        # Unregister from bus
        if self.bus:
            self.bus.unregister_agent(self.id)

        self._running = False
        print(f"⏹️  Agent {self.name} stopped")

    def receive_message(self, message: Message):
        """Receive a message (called by SynapticBus)"""
        self._message_queue.put(message)
        print(f"📬 Agent {self.name} received message: {str(message.content)[:50]}{'...' if len(str(message.content)) > 50 else ''}")

    @abstractmethod
    def process_message(self, message: Message):
        pass

    def _processing_loop(self):
        print(f"🔄 Agent {self.name} processing loop started")

        while not self._stop_event.is_set():
            try:
                message = self._message_queue.get(timeout=0.1)

                start_time = time.time()
                self.process_message(message)
                processing_time = time.time() - start_time

                self._metrics.update_processing_time(processing_time)
                self._message_queue.task_done()

                print(f"✅ Agent {self.name} processed message in {processing_time:.3f}s")

            except queue.Empty:
                continue
            except Exception as e:
                print(f"❌ Error processing message in {self.name}: {e}")
                self._metrics.error_count += 1

        print(f"🛑 Agent {self.name} processing loop stopped")

    def get_metrics(self) -> Dict[str, Any]:
        return {
            "message_count": self._metrics.message_count,
            "processing_time": self._metrics.processing_time,
            "error_count": self._metrics.error_count,
            "last_active": self._metrics.last_active,
            "average_processing_time": (
                self._metrics.processing_time / max(self._metrics.message_count, 1)
            ),
            "subscribed_topics": list(self._subscribed_topics),
            "connected_to_bus": self.bus is not None
        }

class ChatAgent(BaseAgent):
    """
    A social agent that can participate in group conversations

    This agent demonstrates multi-agent communication patterns including
    direct messaging, topic subscriptions, and group discussions.
    """

    def __init__(self, agent_id: Optional[AgentID] = None, name: str = "",
                 personality: str = "friendly", bus: Optional[SynapticBus] = None):
        super().__init__(agent_id, name, memory_capacity=15, bus=bus)
        self.personality = personality
        self.conversation_partners: Set[AgentID] = set()

        # Auto-subscribe to general chat topic
        if self.bus:
            self.subscribe_to_topic("general_chat")
            self.subscribe_to_topic("announcements")

    def process_message(self, message: Message):
        """Process incoming messages with personality-based responses"""
        print(f"🎯 ChatAgent {self.name} processing message from {message.sender}")

        # Remember this interaction
        self.working_memory.store(
            f"Received from {message.sender}: {message.content}",
            importance=1.0
        )

        # Track conversation partner
        self.conversation_partners.add(message.sender)

        # Get message type from metadata
        message_type = message.metadata.get('message_type', MessageType.REQUEST.value)

        # Generate response based on message type and personality
        response = self._generate_response(message.content, message_type, message.sender)

        if response:
            # Determine response type
            if message_type == MessageType.BROADCAST.value:
                # Respond to broadcasts publicly if we have something to say
                topic = message.metadata.get('topic', 'general_chat')
                self.broadcast_to_topic(topic, response, MessagePriority.NORMAL)
            else:
                # Direct response to sender
                self.send_message_via_bus(
                    recipients=[message.sender],
                    content=response,
                    message_type=MessageType.RESPONSE
                )

    def _generate_response(self, content: str, message_type: str, sender: AgentID) -> Optional[str]:
        """Generate personality-based responses"""
        content_lower = str(content).lower()

        # Get recent conversation context
        context = self.working_memory.get_context(3)

        # Personality-based response generation
        if self.personality == "friendly":
            return self._friendly_response(content_lower, message_type, sender, context)
        elif self.personality == "analytical":
            return self._analytical_response(content_lower, message_type, sender, context)
        elif self.personality == "helpful":
            return self._helpful_response(content_lower, message_type, sender, context)
        else:
            return f"I received your message: {content}"

    def _friendly_response(self, content: str, message_type: str, sender: AgentID, context: List) -> Optional[str]:
        """Generate friendly, social responses"""
        if "hello" in content or "hi" in content:
            return f"Hello there! Nice to meet you! 😊"
        elif "how are you" in content:
            return f"I'm doing great, thanks for asking! How about you?"
        elif "?" in content:
            return f"That's a great question! I love chatting about things like this."
        elif message_type == MessageType.BROADCAST.value:
            if len(context) > 2:  # Only respond occasionally to broadcasts
                return None
            return f"Interesting! Thanks for sharing that."
        else:
            return f"Thanks for telling me that! I enjoy our conversation."

    def _analytical_response(self, content: str, message_type: str, sender: AgentID, context: List) -> Optional[str]:
        """Generate analytical, thoughtful responses"""
        if "?" in content:
            return f"Let me think about that... Based on the information available, I would say this requires careful analysis."
        elif any(word in content for word in ["data", "analysis", "problem", "solution"]):
            return f"That's an interesting analytical challenge. What specific aspects are you most concerned about?"
        elif message_type == MessageType.BROADCAST.value:
            return None  # Analytical agents don't respond much to broadcasts
        else:
            return f"I see. Let me process this information and consider the implications."

    def _helpful_response(self, content: str, message_type: str, sender: AgentID, context: List) -> Optional[str]:
        """Generate helpful, service-oriented responses"""
        if "help" in content or "?" in content:
            return f"I'd be happy to help! What specifically can I assist you with?"
        elif "thank" in content:
            return f"You're very welcome! Feel free to ask if you need anything else."
        elif "problem" in content or "issue" in content:
            return f"I understand you're facing a challenge. Let me see how I can help you solve this."
        elif message_type == MessageType.BROADCAST.value:
            if any(word in content for word in ["help", "question", "problem"]):
                return f"If anyone needs assistance with this, I'm here to help!"
            return None
        else:
            return f"Thanks for sharing! Is there anything I can help you with related to this?"

    def initiate_conversation(self, target_agent_id: AgentID, topic: str):
        """Start a conversation with another agent"""
        greeting = f"Hi! I wanted to talk to you about {topic}. What do you think?"

        self.send_message_via_bus(
            recipients=[target_agent_id],
            content=greeting,
            message_type=MessageType.REQUEST
        )

        print(f"💬 {self.name} initiated conversation about '{topic}'")

# =============================================================================
# INITIALIZATION COMPLETE - CLASSES LOADED SUCCESSFULLY
# =============================================================================

print("🔧 Tutorial 3 initialization complete!")
print("✅ All classes loaded successfully:")
print("   - MessagePriority enum")
print("   - Message dataclass")
print("   - MessageType enum")
print("   - AgentMetrics class")
print("   - SimpleWorkingMemory class")
print("   - SynapticBus class")
print("   - BaseAgent class")
print("   - ChatAgent class")
print()
print("🚀 Ready to start the demo!")
print()


🔧 Tutorial 3 initialization complete!
✅ All classes loaded successfully:
   - MessagePriority enum
   - Message dataclass
   - MessageType enum
   - AgentMetrics class
   - SimpleWorkingMemory class
   - SynapticBus class
   - BaseAgent class
   - ChatAgent class

🚀 Ready to start the demo!



In [10]:
# DEMO SECTION: Let's create a multi-agent conversation!
# =============================================================================

print("=" * 60)
print("🚀 Tutorial 3: Agent Communication - Making Agents Talk")
print("=" * 60)
print()

# Step 1: Create the SynapticBus
print("📝 Step 1: Creating the SynapticBus...")
message_bus = SynapticBus(max_queue_size=100)
message_bus.start()
print()

🚀 Tutorial 3: Agent Communication - Making Agents Talk

📝 Step 1: Creating the SynapticBus...
🚌 SynapticBus initialized with max queue size: 100
🔄 SynapticBus routing loop started
▶️  SynapticBus started routing messages



In [11]:
# Step 2: Create multiple agents with different personalities
print("📝 Step 2: Creating multiple ChatAgents with different personalities...")
agents = []

alice = ChatAgent(name="Alice", personality="friendly", bus=message_bus)
bob = ChatAgent(name="Bob", personality="analytical", bus=message_bus)
charlie = ChatAgent(name="Charlie", personality="helpful", bus=message_bus)

agents = [alice, bob, charlie]

# Start all agents
for agent in agents:
    agent.start()

print()

📝 Step 2: Creating multiple ChatAgents with different personalities...
🤖 Initialized agent: Alice (83841ec3...)
📝 Registered agent: Alice (83841ec3...)
📢 Agent Alice subscribed to topic: 'general_chat'
📢 Agent Alice subscribed to topic: 'announcements'
🤖 Initialized agent: Bob (59e34180...)
📝 Registered agent: Bob (59e34180...)
📢 Agent Bob subscribed to topic: 'general_chat'
📢 Agent Bob subscribed to topic: 'announcements'
🤖 Initialized agent: Charlie (c9b22a50...)
📝 Registered agent: Charlie (c9b22a50...)
📢 Agent Charlie subscribed to topic: 'general_chat'
📢 Agent Charlie subscribed to topic: 'announcements'
🔄 Agent Alice processing loop started
▶️  Agent Alice started
🔄 Agent Bob processing loop started
▶️  Agent Bob started
🔄 Agent Charlie processing loop started
▶️  Agent Charlie started



In [12]:
# Step 3: Test direct communication
print("📝 Step 3: Testing direct agent-to-agent communication...")
print()

# Alice initiates conversation with Bob
alice.initiate_conversation(bob.id, "machine learning")
time.sleep(0.5)

# Bob responds by asking Charlie for help
bob.send_message_via_bus(
    recipients=[charlie.id],
    content="Alice is asking about machine learning. Can you help us discuss this?",
    message_type=MessageType.REQUEST
)
time.sleep(0.5)

# Charlie responds to both
charlie.send_message_via_bus(
    recipients=[alice.id, bob.id],
    content="I'd love to help! Machine learning is about training algorithms to find patterns in data.",
    message_type=MessageType.RESPONSE
)
time.sleep(0.5)

print()

📝 Step 3: Testing direct agent-to-agent communication...

📬 Routed message from Alice to 1 recipient(s)
📤 Agent Alice sent via bus: Hi! I wanted to talk to you about machine learning...
💬 Alice initiated conversation about 'machine learning'
📬 Agent Bob received message: Hi! I wanted to talk to you about machine learning...
🎯 ChatAgent Bob processing message from 83841ec3-98d6-46d8-90e5-cda4528f683d
📬 Routed message from Bob to 1 recipient(s)
📤 Agent Bob sent via bus: Let me think about that... Based on the informatio...
✅ Agent Bob processed message in 0.000s
📬 Agent Alice received message: Let me think about that... Based on the informatio...
🎯 ChatAgent Alice processing message from 59e34180-9a58-4393-9267-5eb8e4f5eb0b
📬 Routed message from Alice to 1 recipient(s)
📤 Agent Alice sent via bus: Hello there! Nice to meet you! 😊
✅ Agent Alice processed message in 0.000s
📬 Agent Bob received message: Hello there! Nice to meet you! 😊
🎯 ChatAgent Bob processing message from 83841ec3-98d6-46

In [13]:
# Step 4: Test topic-based broadcasting
print("📝 Step 4: Testing topic-based broadcasting...")
print()

# Alice makes an announcement
alice.broadcast_to_topic(
    "announcements",
    "Hey everyone! I just learned something cool about neural networks!",
    MessagePriority.HIGH
)
time.sleep(0.5)

# Bob shares analysis in general chat
bob.broadcast_to_topic(
    "general_chat",
    "I've been analyzing the latest trends in AI development. The pace of change is accelerating.",
    MessagePriority.NORMAL
)
time.sleep(0.5)

# Charlie offers help publicly
charlie.broadcast_to_topic(
    "general_chat",
    "If anyone has questions about AI or needs help with coding, just let me know!",
    MessagePriority.NORMAL
)
time.sleep(1.0)

print()


📬 Agent Alice received message: Thanks for sharing! Is there anything I can help y...🎯 ChatAgent Alice processing message from c9b22a50-4999-4bc9-af40-0bb6ad613695
📬 Routed message from Alice to 1 recipient(s)
📤 Agent Alice sent via bus: Hello there! Nice to meet you! 😊
✅ Agent Alice processed message in 0.000s

📬 Agent Bob received message: Hello there! Nice to meet you! 😊
📬 Agent Charlie received message: Let me think about that... Based on the informatio...
🎯 ChatAgent Charlie processing message from 59e34180-9a58-4393-9267-5eb8e4f5eb0b
📬 Routed message from Charlie to 1 recipient(s)
📤 Agent Charlie sent via bus: Thanks for sharing! Is there anything I can help y...
✅ Agent Charlie processed message in 0.000s
🎯 ChatAgent Bob processing message from 83841ec3-98d6-46d8-90e5-cda4528f683d
📬 Routed message from Bob to 1 recipient(s)
📤 Agent Bob sent via bus: I see. Let me process this information and conside...
✅ Agent Bob processed message in 0.000s
📬 Agent Alice received message: I see

In [14]:
# Step 5: Check bus statistics
print("📝 Step 5: Checking SynapticBus statistics...")
bus_stats = message_bus.get_system_stats()
print("   Bus Statistics:")
for key, value in bus_stats.items():
    print(f"     {key}: {value}")
print()

📬 Agent Alice received message: That's an interesting analytical challenge. What s...
📬 Agent Bob received message: Hello there! Nice to meet you! 😊
📬 Agent Charlie received message: I'd be happy to help! What specifically can I assi...
📬 Routed message from Charlie to 3 recipient(s)
📢 Charlie broadcast to topic 'general_chat': 3 subscribers
✅ Agent Charlie processed message in 0.031s
🎯 ChatAgent Charlie processing message from 83841ec3-98d6-46d8-90e5-cda4528f683d
✅ Agent Charlie processed message in 0.000s
🎯 ChatAgent Charlie processing message from 59e34180-9a58-4393-9267-5eb8e4f5eb0b
✅ Agent Charlie processed message in 0.000s
🎯 ChatAgent Charlie processing message from c9b22a50-4999-4bc9-af40-0bb6ad613695
📬 Agent Alice received message: Hello there! Nice to meet you! 😊
📬 Agent Bob received message: I'd be happy to help! What specifically can I assi...
📬 Agent Charlie received message: Let me think about that... Based on the informatio...
📬 Routed message from Bob to 3 recipient(s)


In [15]:
# Step 6: Check individual agent information
print("📝 Step 6: Checking individual agent information...")
for agent in agents:
    agent_info = message_bus.get_agent_info(agent.id)
    print(f"   Agent {agent.name}:")
    for key, value in agent_info.items():
        print(f"     {key}: {value}")

    # Also show agent's own metrics
    metrics = agent.get_metrics()
    print(f"     message_count: {metrics['message_count']}")
    print(f"     conversation_partners: {len(agent.conversation_partners)}")
    print()


📬 Agent Alice received message: If anyone needs assistance with this, I'm here to ...
📬 Agent Bob received message: That's a great question! I love chatting about thi...
📬 Agent Charlie received message: That's a great question! I love chatting about thi...
📬 Routed message from Charlie to 3 recipient(s)
📢 Charlie broadcast to topic 'general_chat': 3 subscribers
✅ Agent Charlie processed message in 0.031s
🎯 ChatAgent Charlie processing message from 83841ec3-98d6-46d8-90e5-cda4528f683d
📬 Agent Alice received message: That's an interesting analytical challenge. What s...
📬 Agent Bob received message: I'd be happy to help! What specifically can I assi...
📬 Agent Charlie received message: I'd be happy to help! What specifically can I assi...
📬 Routed message from Bob to 3 recipient(s)
📢 Bob broadcast to topic 'general_chat': 3 subscribers
✅ Agent Bob processed message in 0.031s
🎯 ChatAgent Bob processing message from 59e34180-9a58-4393-9267-5eb8e4f5eb0b
📬 Agent Alice received message: That

In [16]:
# Step 7: Visualize communication patterns
print("📝 Step 7: Visualizing communication patterns...")

# Get recent message history
recent_messages = message_bus.get_recent_messages(limit=20)

if recent_messages:
    try:
        import plotly.graph_objects as go
        import plotly.express as px
        from plotly.subplots import make_subplots

        # Prepare data for visualization
        senders = []
        recipients_list = []
        timestamps = []
        priorities = []
        message_types = []
        contents = []

        for msg in recent_messages:
            for recipient in msg['recipients']:
                senders.append(msg['sender'][:8])
                recipients_list.append(recipient[:8])
                timestamps.append(time.strftime('%H:%M:%S', time.localtime(msg['timestamp'])))
                priorities.append(msg['priority'])
                message_types.append('REQUEST')  # Simplified for demo
                contents.append(msg['content'])

        # Create communication network visualization
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Communication Network', 'Message Timeline',
                'Message Priority Distribution', 'Agent Activity Levels'
            ),
            specs=[[{"type": "scatter"}, {"type": "scatter"}],
                   [{"type": "histogram"}, {"type": "bar"}]]
        )

        # Network graph of communications
        unique_agents = list(set(senders + recipients_list))
        agent_positions = {agent: i for i, agent in enumerate(unique_agents)}

        # Create edges for the network
        edge_x = []
        edge_y = []
        for sender, recipient in zip(senders, recipients_list):
            edge_x.extend([agent_positions[sender], agent_positions[recipient], None])
            edge_y.extend([0, 1, None])

        fig.add_trace(
            go.Scatter(
                x=edge_x, y=edge_y,
                mode='lines',
                line=dict(width=2, color='lightblue'),
                showlegend=False,
                name='Communications'
            ),
            row=1, col=1
        )

        # Add agent nodes
        fig.add_trace(
            go.Scatter(
                x=[agent_positions[agent] for agent in unique_agents],
                y=[0.5] * len(unique_agents),
                mode='markers+text',
                marker=dict(size=20, color='red'),
                text=unique_agents,
                textposition="middle center",
                name='Agents',
                showlegend=False
            ),
            row=1, col=1
        )

        # Message timeline
        priority_colors = {'LOW': 'green', 'NORMAL': 'blue', 'HIGH': 'orange', 'URGENT': 'red'}
        colors = [priority_colors.get(p, 'blue') for p in priorities]

        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=list(range(len(timestamps))),
                mode='markers',
                marker=dict(size=10, color=colors),
                text=[f"{s} → {r}<br>{c[:30]}..." for s, r, c in zip(senders, recipients_list, contents)],
                hovertemplate='%{text}<br>Time: %{x}<br>Priority: %{marker.color}<extra></extra>',
                name='Messages'
            ),
            row=1, col=2
        )

        # Priority distribution
        fig.add_trace(
            go.Histogram(
                x=priorities,
                name='Priority Distribution',
                marker_color='lightgreen'
            ),
            row=2, col=1
        )

        # Agent activity (message count per agent)
        from collections import Counter
        sender_counts = Counter(senders)

        fig.add_trace(
            go.Bar(
                x=list(sender_counts.keys()),
                y=list(sender_counts.values()),
                name='Messages Sent',
                marker_color='lightcoral'
            ),
            row=2, col=2
        )

        # Update layout
        fig.update_layout(
            title_text="Multi-Agent Communication Analysis",
            showlegend=True,
            height=800,
            template='plotly_white'
        )

        # Update axes
        fig.update_xaxes(title_text="Agent", row=1, col=1)
        fig.update_yaxes(title_text="Connection", row=1, col=1)

        fig.update_xaxes(title_text="Time", row=1, col=2)
        fig.update_yaxes(title_text="Message Sequence", row=1, col=2)

        fig.update_xaxes(title_text="Priority Level", row=2, col=1)
        fig.update_yaxes(title_text="Count", row=2, col=1)

        fig.update_xaxes(title_text="Agent", row=2, col=2)
        fig.update_yaxes(title_text="Messages Sent", row=2, col=2)

        fig.show()

        print("   ✅ Communication analysis visualization created!")
        print("   📊 The plots show:")
        print("      - Network diagram of agent communications")
        print("      - Timeline of messages with priority color coding")
        print("      - Distribution of message priorities")
        print("      - Activity levels showing which agents are most active")
        print()

        # Create a message flow sankey diagram
        fig2 = go.Figure(data=[go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=unique_agents,
                color="blue"
            ),
            link=dict(
                source=[agent_positions[sender] for sender in senders],
                target=[agent_positions[recipient] for recipient in recipients_list],
                value=[1] * len(senders),  # All messages have equal weight
                color="rgba(255,0,255,0.4)"
            )
        )])

        fig2.update_layout(
            title_text="Message Flow Between Agents",
            font_size=10,
            template='plotly_white'
        )

        fig2.show()
        print("   ✅ Message flow diagram created!")
        print()

    except ImportError:
        print("   ⚠️  Plotly not available - skipping visualization")
        print("   💡 To see communication visualizations, install plotly: pip install plotly")
        print("   📊 Communication summary:")
        print(f"      Total messages: {len(recent_messages)}")
        print(f"      Unique senders: {len(set(msg['sender'] for msg in recent_messages))}")
        print("      Recent activity:")
        for msg in recent_messages[-3:]:
            print(f"        {msg['sender'][:8]} → {len(msg['recipients'])} recipients: {msg['content'][:40]}...")
        print()
else:
    print("   ⚠️  No recent messages to visualize")
    print()

📬 Agent Alice received message: If anyone needs assistance with this, I'm here to ...
📬 Agent Bob received message: If anyone needs assistance with this, I'm here to ...
📬 Agent Charlie received message: That's an interesting analytical challenge. What s...
📬 Routed message from Bob to 3 recipient(s)
📢 Bob broadcast to topic 'general_chat': 3 subscribers
✅ Agent Bob processed message in 0.031s
🎯 ChatAgent Bob processing message from 83841ec3-98d6-46d8-90e5-cda4528f683d
✅ Agent Bob processed message in 0.000s
🎯 ChatAgent Bob processing message from c9b22a50-4999-4bc9-af40-0bb6ad613695
📬 Agent Alice received message: Let me think about that... Based on the informatio...
📬 Agent Bob received message: That's an interesting analytical challenge. What s...
📬 Agent Charlie received message: Hello there! Nice to meet you! 😊
📬 Routed message from Alice to 3 recipient(s)
📢 Alice broadcast to topic 'general_chat': 3 subscribers
✅ Agent Alice processed message in 0.031s
🎯 ChatAgent Alice processin

   ✅ Communication analysis visualization created!
   📊 The plots show:
      - Network diagram of agent communications
      - Timeline of messages with priority color coding
      - Distribution of message priorities
      - Activity levels showing which agents are most active

📬 Agent Alice received message: That's an interesting analytical challenge. What s...
📬 Agent Bob received message: That's an interesting analytical challenge. What s...
📬 Agent Charlie received message: That's an interesting analytical challenge. What s...
📬 Routed message from Alice to 3 recipient(s)
📢 Alice broadcast to topic 'general_chat': 3 subscribers
✅ Agent Alice processed message in 0.041s
🎯 ChatAgent Alice processing message from c9b22a50-4999-4bc9-af40-0bb6ad613695
📬 Agent Alice received message: Hello there! Nice to meet you! 😊
📬 Agent Bob received message: Hello there! Nice to meet you! 😊
📬 Agent Charlie received message: Hello there! Nice to meet you! 😊
📬 Routed message from Charlie to 3 recipie

📬 Routed message from Charlie to 3 recipient(s)
📢 Charlie broadcast to topic 'general_chat': 3 subscribers
✅ Agent Charlie processed message in 0.047s
🎯 ChatAgent Charlie processing message from c9b22a50-4999-4bc9-af40-0bb6ad613695
📬 Agent Alice received message: I'd be happy to help! What specifically can I assi...
📬 Agent Bob received message: I'd be happy to help! What specifically can I assi...
📬 Agent Charlie received message: I'd be happy to help! What specifically can I assi...
   ✅ Message flow diagram created!

📬 Routed message from Bob to 3 recipient(s)
📢 Bob broadcast to topic 'general_chat': 3 subscribers
✅ Agent Bob processed message in 0.041s
🎯 ChatAgent Bob processing message from 83841ec3-98d6-46d8-90e5-cda4528f683d
✅ Agent Bob processed message in 0.000s
🎯 ChatAgent Bob processing message from c9b22a50-4999-4bc9-af40-0bb6ad613695


In [17]:
# Step 8: Test advanced communication patterns
print("📝 Step 8: Testing advanced communication patterns...")
print()

# Create a group discussion scenario
print("   Starting group discussion about 'AI Ethics'...")

# Alice starts the discussion
alice.broadcast_to_topic(
    "general_chat",
    "I've been thinking about AI ethics lately. What are everyone's thoughts on AI bias?",
    MessagePriority.HIGH
)
time.sleep(0.3)

# Bob provides analytical perspective
bob.send_message_via_bus(
    recipients=[alice.id],
    content="That's a complex issue. Bias often comes from training data. We need systematic approaches to detect and mitigate it.",
    message_type=MessageType.RESPONSE
)
time.sleep(0.3)

# Charlie offers practical help
charlie.broadcast_to_topic(
    "general_chat",
    "I can help implement bias detection algorithms if anyone wants to work on this together!",
    MessagePriority.NORMAL
)
time.sleep(0.3)

# Alice responds positively
alice.send_message_via_bus(
    recipients=[bob.id, charlie.id],
    content="Great insights! Bob's analysis combined with Charlie's technical skills could make a real difference.",
    message_type=MessageType.RESPONSE
)
time.sleep(0.5)

print("   ✅ Group discussion completed!")
print()

📬 Agent Alice received message: That's a great question! I love chatting about thi...
📬 Agent Bob received message: That's a great question! I love chatting about thi...
📬 Agent Charlie received message: That's a great question! I love chatting about thi...
📬 Routed message from Charlie to 3 recipient(s)
📢 Charlie broadcast to topic 'general_chat': 3 subscribers
✅ Agent Charlie processed message in 0.031s
🎯 ChatAgent Charlie processing message from 83841ec3-98d6-46d8-90e5-cda4528f683d
✅ Agent Charlie processed message in 0.000s
🎯 ChatAgent Charlie processing message from c9b22a50-4999-4bc9-af40-0bb6ad613695
📬 Agent Alice received message: I'd be happy to help! What specifically can I assi...
📬 Agent Bob received message: I'd be happy to help! What specifically can I assi...
📬 Agent Charlie received message: I'd be happy to help! What specifically can I assi...
📬 Routed message from Bob to 3 recipient(s)
📢 Bob broadcast to topic 'general_chat': 3 subscribers
✅ Agent Bob processed messag

In [18]:
# Step 9: Check final system state
print("📝 Step 9: Final system state analysis...")

# Get updated bus statistics
final_stats = message_bus.get_system_stats()
print("   Final Bus Statistics:")
for key, value in final_stats.items():
    print(f"     {key}: {value}")
print()

# Check agent conversation networks
print("   Agent Conversation Networks:")
for agent in agents:
    print(f"     {agent.name}: connected to {len(agent.conversation_partners)} other agents")
    partner_names = []
    for partner_id in agent.conversation_partners:
        partner_info = message_bus.get_agent_info(partner_id)
        if partner_info:
            partner_names.append(partner_info['name'])
    print(f"       Partners: {', '.join(partner_names) if partner_names else 'None'}")
print()


📝 Step 9: Final system state analysis...
   Final Bus Statistics:
     running: True
     registered_agents: 3
     total_queue_size: 300
     delivery_stats: {'delivered': 37169, 'dropped': 46}
     topics: {'general_chat': 3, 'announcements': 3}
     recent_messages: 1000

   Agent Conversation Networks:
     Alice: connected to 3 other agents
       Partners: Alice, Bob, Charlie
     Bob: connected to 3 other agents
       Partners: Alice, Bob, Charlie
     Charlie: connected to 3 other agents
       Partners: Alice, Bob, Charlie

📬 Agent Alice received message: Let me think about that... Based on the informatio...
📬 Agent Bob received message: Let me think about that... Based on the informatio...
📬 Agent Charlie received message: Let me think about that... Based on the informatio...
📬 Routed message from Alice to 3 recipient(s)
📢 Alice broadcast to topic 'general_chat': 3 subscribers
✅ Agent Alice processed message in 0.032s
🎯 ChatAgent Alice processing message from c9b22a50-4999-4

In [19]:
# Step 10: Clean up
print("📝 Step 10: Stopping agents and cleaning up...")

# Stop all agents
for agent in agents:
    agent.stop()

# Stop the message bus
message_bus.stop()
print()

print("✅ Tutorial 3 Complete!")
print()

📬 Agent Alice received message: I'd be happy to help! What specifically can I assi...
📬 Agent Bob received message: I'd be happy to help! What specifically can I assi...
📬 Agent Charlie received message: I'd be happy to help! What specifically can I assi...
📬 Routed message from Bob to 3 recipient(s)
📢 Bob broadcast to topic 'general_chat': 3 subscribers
✅ Agent Bob processed message in 0.031s
🎯 ChatAgent Bob processing message from 83841ec3-98d6-46d8-90e5-cda4528f683d
✅ Agent Bob processed message in 0.000s
🎯 ChatAgent Bob processing message from c9b22a50-4999-4bc9-af40-0bb6ad613695
✅ Agent Bob processed message in 0.000s
🎯 ChatAgent Bob processing message from 59e34180-9a58-4393-9267-5eb8e4f5eb0b
📬 Agent Alice received message: Let me think about that... Based on the informatio...
📬 Agent Bob received message: Let me think about that... Based on the informatio...
📬 Agent Charlie received message: Let me think about that... Based on the informatio...
📬 Routed message from Alice to 3 r

# SUMMARY OF WHAT WE LEARNED
# =============================================================================

print("📚 WHAT WE LEARNED:")


print("=" * 40)

print("1. 🚌 Built a centralized message bus system")

print("   - SynapticBus for routing messages between agents")

print("   - Priority-based message queuing")

print("   - Agent registration and discovery")

print("   - Topic-based subscription system")

print()

print("2. 📡 Enhanced agent communication capabilities")

print("   - Direct agent-to-agent messaging")

print("   - Broadcast messaging to topic subscribers")

print("   - Message types for different communication patterns")

print("   - Automatic bus integration and lifecycle management")

print()

print("3. 🤖 Created personality-based chat agents")

print("   - Different response patterns (friendly, analytical, helpful)")

print("   - Context-aware conversation handling")

print("   - Group discussion and collaboration patterns")

print("   - Social network formation through conversations")

print()

print("4. 📊 Added comprehensive communication monitoring")

print("   - Message delivery statistics and error tracking")

print("   - Agent activity and network analysis")

print("   - Real-time visualization of communication patterns")

print("   - System health and performance monitoring")

print()

# COMMON ERRORS AND SOLUTIONS
# =============================================================================

print("⚠️  COMMON ERRORS AND SOLUTIONS:")


print("=" * 40)

print("1. 🐛 'Agent not connected to bus' errors")

print("   Problem: Trying to send messages without bus connection")

print("   Solution: Always call agent.connect_to_bus() or pass bus in constructor")

print("   Solution: Check agent.bus is not None before sending messages")

print()

print("2. 🐛 'Message queue full' warnings")

print("   Problem: Agent not processing messages fast enough")

print("   Solution: Increase max_queue_size parameter in SynapticBus")

print("   Solution: Optimize agent.process_message() for faster processing")

print("   Solution: Use higher priority for important messages")

print()

print("3. 🐛 'Unknown recipient' warnings")

print("   Problem: Sending messages to unregistered agents")

print("   Solution: Check agent registration with bus.get_agent_info()")

print("   Solution: Use topic broadcasting instead of direct messaging")

print()

print("4. 🐛 Messages not being delivered")

print("   Problem: SynapticBus not started or agents not running")

print("   Solution: Always call bus.start() before sending messages")

print("   Solution: Ensure all recipient agents are started")

print("   Solution: Check bus statistics for delivery errors")

print()

print("5. 🐛 Topic subscription not working")

print("   Problem: Agent not receiving broadcast messages")

print("   Solution: Verify subscription with agent's subscribed_topics list")

print("   Solution: Check topic name spelling (case sensitive)")

print("   Solution: Ensure agent is registered before subscribing")

print()

print("6. 🐛 High memory usage with many agents")

print("   Problem: Message queues and history growing unbounded")

print("   Solution: Set appropriate queue size limits")

print("   Solution: Regularly clean up old message history")

print("   Solution: Unregister inactive agents")

print()

print("🎉 Ready for Tutorial 4: Simple Reflex Rules!")

print("   Next we'll add intelligent behavior rules to our agents...")

---

🔒 **INTELLECTUAL PROPERTY & LICENSE NOTICE**

This tutorial and its contents — including code, architecture, narrative examples, and educational structure — are the intellectual property of **Shalini Ananda, PhD** and part of the **Neuron Framework** under a **Modified MIT License with Attribution**.

- Commercial use, redistribution, or derivative works **must** include clear and visible attribution to the original author.
- Use in products, consulting engagements, or educational materials **must reference this repository and author name.**
- Removal of author credit or misrepresentation of origin constitutes **a violation of the license and may trigger legal action.**
- You may **not white-label, obfuscate, or rebrand** this work without explicit, written permission.

Use of this tutorial in Colab or any other platform implies agreement with these terms.

📘 **License**: [LICENSE.md](../LICENSE.md)  
📌 **Notice**: [NOTICE.md](../NOTICE.md)  
🧠 **Author**: [Shalini Ananda, PhD](https://github.com/ShaliniAnandaPhD)

---

